In [1]:
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pygeohydro as gh
import shutil
from matplotlib.font_manager import fontManager
from tqdm.auto import tqdm

Get the data from the [National Inventory of Dams](https://nid.sec.usace.army.mil) using [PyGeoHydro](https://github.com/cheginit/pygeohydro) package.

In [2]:
try:
    with open("input_data/nid.pkl", "rb") as f:
        nid = pickle.load(f)
except FileNotFoundError:
    nid = gh.get_nid()

Filter the data to only include dams that are within Contiguous US.

In [3]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
conus = world[world.name == "United States of America"].geometry.iloc[0][0]
conus = gpd.GeoDataFrame({"FID": [0], "geometry": conus}, crs=world.crs)
nid_conus = nid[nid.within(conus.geometry.unary_union)]

Plot the frames.

In [4]:
column = "DAM_HEIGHT"
cmap = "viridis"
min_q, max_q = 0.1, 0.9

label = f"{column.replace('_', ' ').title()} ({nid.attrs[column].split(',', 1)[0][1:].lower()})"
label = "\n".join([label, f"{min_q} - {max_q} Quantile"])
norm = plt.Normalize(vmin=nid_conus[column].quantile(min_q), vmax=nid_conus[column].quantile(max_q))

dpi = 250.0
figsize = (1920.0/dpi, 1080.0/dpi)
font = 'Lato'
if font in set([f.name for f in fontManager.ttflist]):
    matplotlib.rcParams['font.sans-serif'] = font
    matplotlib.rcParams['font.family'] = "sans-serif"
plt.ioff()

def get_ax():
    ax = conus.plot(figsize=figsize, facecolor="none", edgecolor="k")
    ax.axis(False)
    fig = ax.figure
    fig.set_dpi(dpi)
    cax = fig.add_axes(
        [
            ax.get_position().x1 + 0.01,
            ax.get_position().y0,
            0.02,
            ax.get_position().height,
        ]
    )
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    fig.colorbar(sm, cax=cax, label=label)
    return ax
    
years = range(nid_conus.YEAR_COMPLETED.min() + 1, nid_conus.YEAR_COMPLETED.max() + 1)
with tqdm(total=len(years), desc="Plotting") as pbar:
    for year in years:
        pbar.set_postfix_str(f"Year: {year}")
        dams = nid_conus[nid_conus.YEAR_COMPLETED <= year]
        ax = get_ax()
        dams.plot(
            ax=ax,
            column=column,
            cmap=cmap,
            norm=norm,
            alpha=0.3,
            markersize=3,
        )
        ax.set_title(f"Dams Completed Up to {year}\nTotal = {len(dams):,}")
        ax.annotate(
            f"Largest (height): {dams[column].max():.1f} ft", xy=(0, 0), xycoords=ax.transAxes
        )
        ax.figure.savefig(f"tmp/{year}.png", bbox_inches="tight", dpi=dpi, facecolor="w")
        plt.close("all")
        pbar.update(1)

Plotting:   0%|          | 0/378 [00:00<?, ?it/s]

Repeate the last frame 100 times.

In [5]:
for i in range(1, 100):
    shutil.copy(f"tmp/{years[-1]}.png", f"tmp/{years[-1] + i}.png")

Convert the frames to a video file.

In [6]:
!ffmpeg -hide_banner -loglevel panic -start_number 1641 -i tmp/%04d.png -pix_fmt yuv420p -vf scale=1920:-2 -y input_data.mp4